## 1. Define a Specification

In the first phase of SDMT, we define a `Specification` that represents the requirements the completed model must meet in order to be acceptable for use in the system into which it will be integrated.

#### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces.

In [1]:
import os
from mlte.session import set_context, set_store

store_path = os.path.join(os.getcwd(), "store")
os.makedirs(
    store_path, exist_ok=True
)  # Ensure we are creating the folder if it is not there.

set_context("IrisClassifier", "0.0.1")
set_store(f"local://{store_path}")

#### Build a `Specification`

In MLTE, we define requirements by constructing a specification (`Spec`). For each property, we define the validations to perform as well.

In [2]:
from mlte.spec.spec import Spec

from mlte.property.costs.storage_cost import StorageCost
from mlte.property.costs.training_memory_cost import TrainingMemoryCost
from mlte.property.costs.training_compute_cost import TrainingComputeCost
from mlte.property.functionality.task_efficacy import TaskEfficacy


from mlte.measurement.storage import LocalObjectSize
from mlte.measurement.cpu import LocalProcessCPUUtilization
from mlte.measurement.memory import LocalProcessMemoryConsumption
from confusion_matrix import ConfusionMatrix
from mlte.value.types.real import Real
from mlte.value.types.image import Image

spec = Spec(
    properties={
        TaskEfficacy(
            "Important to understand if the model is useful for this case"
        ): {
            "accuracy": Real.greater_or_equal_to(0.98),
            "confusion matrix": ConfusionMatrix.misclassification_count_less_than(
                2
            ),
            "class distribution": Image.ignore("Inspect the image."),
        },
        StorageCost("Critical since model will be in an embedded decice"): {
            "model size": LocalObjectSize.value().less_than(3000)
        },
        TrainingMemoryCost("Useful to evaluate resources needed"): {
            "training memory": LocalProcessMemoryConsumption.value().average_consumption_less_than(
                60000
            )
        },
        TrainingComputeCost("Useful to evaluate resources needed"): {
            "training cpu": LocalProcessCPUUtilization.value().max_utilization_less_than(
                5.0
            )
        },
    }
)
spec.save(parents=True, force=True)